# Boxcars Gem Notebook
Below you will find some sample boxcar gem code fragments.

## Env Setup
Before we get started, install the gems we need here:
```bash
gem install boxcars dotenv
```
and then create / edit .env to have SERPAPI_API_KEY and OPENAI_ACCESS_TOKEN

In [1]:
# setup for code below
require "dotenv/load"
require "boxcars"

true

## Examples
Here are several examples of using the boxcar gems. First we will start with individual Boxcars, and then we will move on to using Trains.

### Search using the Google Serp API (set SERPAPI_API_KEY in .env)

In [3]:
# showcase Google Serp search
s = Boxcars::GoogleSearch.new
s.run("what temperature is it in Phoenix?")

Question: what temperature is it in Phoenix?
Answer: High 64F


"High 64F"

### Calculator that uses ruby for hard math

In [2]:
c = Boxcars::Calculator.new
c.run("what is pi to the third power times epsilon?")

> Entering Calculator#run
what is pi to the third power times epsilon?
RubyREPL: puts(Math::PI ** 3 * Math::E)
Answer: 84.28379846823243

84.28379846823243
< Exiting Calculator#run


"84.28379846823243"

### Active Record Example using SQL code generation

In [3]:
# first stub out an in memory sqlite3 and a simple helpdesk example
require 'active_record'
require './helpdesk_sample'

-- create_table("users", {:force=>:cascade})
   -> 0.0083s
-- create_table("comments", {:force=>:cascade})
   -> 0.0004s
-- create_table("tickets", {:force=>:cascade})
   -> 0.0005s


true

In [5]:
# get the answer
boxcar = Boxcars::SQL.new
boxcar.run("how many open tickets for John?")

> Entering data#run
how many open tickets for John?
SELECT COUNT(*) FROM tickets WHERE user_id = (SELECT id FROM users WHERE name = 'John') AND status = 0;
Answer: [{"COUNT(*)"=>1}]
Answer: [{"COUNT(*)"=>1}]
< Exiting data#run


"Answer: [{\"COUNT(*)\"=>1}]"

In [6]:
boxcar.run("how many comments do we have on closed tickets?")

> Entering data#run
how many comments do we have on closed tickets?
SELECT COUNT(*) FROM comments INNER JOIN tickets ON comments.ticket_id = tickets.id WHERE tickets.status = 0;
Answer: [{"COUNT(*)"=>4}]
Answer: [{"COUNT(*)"=>4}]
< Exiting data#run


"Answer: [{\"COUNT(*)\"=>4}]"

### Active Record Example using ActiveRecord code generation

In [8]:
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
helpdesk.run("how many comments do we have on open tickets?")

> Entering helpdesk#run
how many comments do we have on open tickets?
Ticket.where(status: 0).joins(:comments).count
Answer: 4
< Exiting helpdesk#run


"Answer: 4"

## Putting it all together - Trains
Trains uses a series of Boxcars to compute an answer by composing answers from the union of queries to individual boxcars as needed.
### a one car train
This is similar to just running Boxcar.run

In [9]:
c = Boxcars::Calculator.new
train = Boxcars.train.new(boxcars: [c])
train.run "what is pi squared?"

> Entering Zero Shot#run
what is pi squared?
> Entering Calculator#run
pi squared
Answer: 9.869604401089358
< Exiting Calculator#run
Observation: Answer: 9.869604401089358
I now know the final answer.
Final Answer: 9.869604401089358
< Exiting Zero Shot#run


"9.869604401089358"

### a two car train
Here we have a HelpDesk ActiveRecord database and a calculator at our disposal to get an answer

In [21]:
c = Boxcars::Calculator.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.default_train.new(boxcars: [c, helpdesk])
train.run "the number of helpdesk comments by John for open tickets times 2 pi?"

> Entering Zero Shot#run
the number of helpdesk comments by John for open tickets times 2 pi?
> Entering helpdesk#run
Find the number of helpdesk comments by John for open tickets
Comment.where(user_id: User.where(name: 'John'), ticket_id: Ticket.where(status: 0)).count
Answer: 2
< Exiting helpdesk#run
Observation: Answer: 2
> Entering Calculator#run
2 * 2pi
RubyREPL: puts(2 * 2 * Math::PI)
Answer: 12.566370614359172

12.566370614359172
< Exiting Calculator#run
Observation: 12.566370614359172
I now know the final answer
Final Answer: 12.566370614359172
< Exiting Zero Shot#run


"12.566370614359172"